In [1]:
import sys,os
sys.path.append('../../')
from smodels.base import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'
import numpy as np
from smodels.decomposition import decomposer
from smodels.base.physicsUnits import fb, GeV, TeV
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.matching.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.base.smodelsLogging import setLogLevel
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.tools.coverage import Uncovered,UncoveredGroup
setLogLevel("error")
%load_ext line_profiler
# Set the path to the database
database = Database(os.path.expanduser("../database_extra"))

### Load model

In [2]:
slhafile = '../../inputFiles/slha/longLived.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 0.1*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               massCompress=True, invisibleCompress=True,
                               minmassgap=mingap)

In [4]:
print(len(topDict.getSMSList()))
totWeight = None
for el in topDict.getSMSList():
    if totWeight is None:
        totWeight = el.weightList.copy()
    else:
        totWeight += el.weightList
print(totWeight.getMaxXsec())

51
2.34E-02 [pb]


### Load the experimental results to be used.

In [5]:
listOfExpRes = database.getExpResults()

### Compute the theory predictions

In [6]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = []
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, topDict, combinedResults=False, marginalize=False)
    if not predictions:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % expResult.globalInfo.id)
    for theoryPrediction in predictions:
        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)


 Theory Predictions and Constraints:

 ATLAS-SUSY-2018-12 
r = 8.233E-01

 ATLAS-SUSY-2018-12 
r = 6.374E-01

 CMS-SUS-16-036 
r = 4.051E+00
r = 4.723E-01


In [7]:
# import time
# t0 = time.time()
uncovered = Uncovered(topDict, sqrts=13.*TeV)
# print(time.time()-t0)

In [8]:
# First sort coverage groups by label
groups = sorted(uncovered.groups[:], key=lambda g: g.label)
# Print uncovered cross-sections:
for group in groups:
    print("\nTotal cross-section for %s (fb): %10.3E\n" % (group.description, group.getTotalXSec()))


Total cross-section for missing topologies (fb):  5.710E+00


Total cross-section for missing topologies with displaced decays (fb):  5.719E-03


Total cross-section for missing topologies with prompt decays (fb):  8.648E+00


Total cross-section for topologies outside the grid (fb):  2.943E+00



In [9]:
missingTopos = uncovered.getGroup('missing (prompt)')
finalStates = {}
for iel,genEl in enumerate(missingTopos.finalStateSMS[:]):
    elStr = str(genEl)
    if elStr not in finalStates:
        finalStates[elStr] = genEl.missingX
    else:
        finalStates[elStr] += genEl.missingX

            # Print some of the missing topologies:
print(len(finalStates))
for fs in sorted(finalStates.keys()):
    print('Element:', fs)
    print('\tcross-section (fb):', np.round(finalStates[fs],4))

17
Element: (PV > HSCP,HSCP)
	cross-section (fb): 0.8032
Element: (PV > HSCP,MET,jet,jet,ta,W)
	cross-section (fb): 0.1489
Element: (PV > HSCP,MET,l,nu,nu)
	cross-section (fb): 0.0
Element: (PV > HSCP,MET,ta,W)
	cross-section (fb): 0.1309
Element: (PV > MET,MET)
	cross-section (fb): 0.1669
Element: (PV > MET,MET,jet)
	cross-section (fb): 0.3509
Element: (PV > MET,MET,jet,b,b)
	cross-section (fb): 0.206
Element: (PV > MET,MET,jet,jet,jet)
	cross-section (fb): 1.1117
Element: (PV > MET,MET,jet,jet,l)
	cross-section (fb): 0.3755
Element: (PV > MET,MET,jet,jet,l,nu)
	cross-section (fb): 0.319
Element: (PV > MET,MET,jet,jet,ta)
	cross-section (fb): 0.1113
Element: (PV > MET,MET,jet,t,b,W)
	cross-section (fb): 0.3604
Element: (PV > MET,MET,jet,t,t)
	cross-section (fb): 1.9273
Element: (PV > MET,MET,jet,t,t,Z)
	cross-section (fb): 0.2103
Element: (PV > MET,MET,l)
	cross-section (fb): 1.113
Element: (PV > MET,MET,l,l)
	cross-section (fb): 1.3126
Element: (PV > MET,MET,l,l,nu,nu,nu,nu)
	cross-s

In [10]:
missingDisplaced = uncovered.getGroup('missing (displaced)')
# Print elements with displaced decays:
finalStates = {}
for iel,genEl in enumerate(missingDisplaced.finalStateSMS[:]):
    elStr = str(genEl)
    if elStr not in finalStates:
        finalStates[elStr] = genEl.missingX
    else:
        finalStates[elStr] += genEl.missingX

            # Print some of the missing topologies:
print(len(finalStates))
for fs in sorted(finalStates.keys()):
    print('Element:', fs)
    print('\tcross-section (fb):', np.round(finalStates[fs],4))

2
Element: (PV > HSCP,MET,l,nu,nu)
	cross-section (fb): 0.0057
Element: (PV > MET,MET,l,l,nu,nu,nu,nu)
	cross-section (fb): 0.0


In [11]:
missingGrid = uncovered.getGroup('outsideGrid (all)')
# Print elements with displaced decays:
finalStates = {}
for iel,genEl in enumerate(missingGrid.finalStateSMS[:]):
    elStr = str(genEl)
    if elStr not in finalStates:
        finalStates[elStr] = genEl.missingX
    else:
        finalStates[elStr] += genEl.missingX

            # Print some of the missing topologies:
print(len(finalStates))
for fs in sorted(finalStates.keys()):
    print('Element:', fs)
    print('\tcross-section (fb):', np.round(finalStates[fs],4))

4
Element: (PV > MET,MET,jet)
	cross-section (fb): 0.3509
Element: (PV > MET,MET,l)
	cross-section (fb): 1.113
Element: (PV > MET,MET,l,l)
	cross-section (fb): 1.3126
Element: (PV > MET,MET,nu,nu)
	cross-section (fb): 0.1669
